In [1]:
from dijkstra import Dijkstra
from routing import select_path, shortest_path_from_first
from knn import knn

from concrete import fhe

In [2]:
def reconstruct_path(previous):
    path = []
    current = len(previous) - 1
    while current > 0:
        path.append(current)
        current = previous[current]
    path.append(0)
    return path[::-1]

In [3]:
# 9 represents inf
inf = 9
weights = [[0, 1, 2, inf],
           [1, 0, 3, 3],
           [2, 3, 0, 2],
           [inf, 3, 2, 0]]

In [4]:
# Shortest-path with hidden graph
compiler = fhe.Compiler(shortest_path_from_first, {"weights": "encrypted"})
shortest_path_from_first_circuit = compiler.compile([(weights)])

In [5]:
shortest_path_from_first_circuit.encrypt_run_decrypt(weights)

array([0, 0, 0, 1], dtype=uint64)

In [6]:
reconstruct_path(shortest_path_from_first_circuit.encrypt_run_decrypt(weights))

[0, 1, 3]

In [7]:
# Server's map data
dijkstra = Dijkstra(weights)
paths = dijkstra.get_all_shortest_paths()
paths

array([[[0, 0, 0],
        [0, 1, 0],
        [0, 2, 0],
        [0, 1, 3]],

       [[1, 0, 1],
        [1, 1, 1],
        [1, 2, 1],
        [1, 3, 1]],

       [[2, 0, 2],
        [2, 1, 2],
        [2, 2, 2],
        [2, 3, 2]],

       [[3, 2, 0],
        [3, 1, 3],
        [3, 2, 3],
        [3, 3, 3]]])

In [8]:
# Client's selection
start = [1, 0, 0, 0]
end = [0, 0, 0, 1]

In [9]:
compiler = fhe.Compiler(select_path, {"paths": "clear", "origin": "encrypted", "destination": "encrypted"})
select_path_circuit = compiler.compile([(paths, start, end)])

In [10]:
select_path_circuit.encrypt_run_decrypt(paths, start, end)

array([0, 1, 3], dtype=uint64)

# KNN

In [11]:
distances, _ = dijkstra.shortest_from_origin(1)
distances = distances.astype(int)
distances

array([1, 0, 3, 3])

In [12]:
knn(distances)

array([1, 1, 0, 0])

In [13]:
compiler = fhe.Compiler(knn, {"distances": "encrypted"})
knn_circuit = compiler.compile([distances])

In [14]:
knn_circuit.encrypt_run_decrypt(distances)

array([1, 1, 0, 0], dtype=uint64)